In [1]:
import pandas as pd


In [2]:
import numpy as np

In [3]:
!pip install pycryptodome



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.6 MB/s eta 0:00:00


In [4]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode


In [5]:
def setup_group():
    key = RSA.generate(2048)  # Generate a 2048-bit RSA key pair
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return public_key, private_key

In [6]:
def encrypt_data(data, public_key):
    # Generate a random symmetric key (AES-256) for data encryption
    symmetric_key = get_random_bytes(32)

    # Encrypt data using AES
    cipher_aes = AES.new(symmetric_key, AES.MODE_GCM)
    ciphertext, tag = cipher_aes.encrypt_and_digest(data.encode())

    # Encrypt the symmetric key with RSA public key
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_symmetric_key = cipher_rsa.encrypt(symmetric_key)

    # Return ciphertext, RSA-encrypted AES key, tag, and nonce
    return b64encode(ciphertext).decode(), b64encode(encrypted_symmetric_key).decode(), b64encode(tag).decode(), b64encode(cipher_aes.nonce).decode()

In [7]:
def decrypt_data(ciphertext, encrypted_symmetric_key, tag, nonce, private_key):
    # Import the private RSA key
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)

    # Decrypt the symmetric key with RSA private key
    symmetric_key = cipher_rsa.decrypt(b64decode(encrypted_symmetric_key))

    # Decrypt the data using AES
    cipher_aes = AES.new(symmetric_key, AES.MODE_GCM, nonce=b64decode(nonce))
    decrypted_data = cipher_aes.decrypt_and_verify(b64decode(ciphertext), b64decode(tag))
    return decrypted_data.decode()

In [8]:
if __name__ == "__main__":
    # Step 1: Setup group keys
    public_key, private_key = setup_group()

    # Data to be shared
    data = "Confidential data for group members only."

    # Encrypt data for the group
    ciphertext, encrypted_symmetric_key, tag, nonce = encrypt_data(data, public_key)
    print("Encrypted Data:", ciphertext)
    print("Encrypted Symmetric Key:", encrypted_symmetric_key)
    print("Tag:", tag)
    print("Nonce:", nonce)

    # Decrypt data as a group member with access to the private key
    decrypted_data = decrypt_data(ciphertext, encrypted_symmetric_key, tag, nonce, private_key)
    print("Decrypted Data:", decrypted_data)

Encrypted Data: ZeLrdKLM9vc8V6s8Z7WIwFSHmrWKGE7w7o4irQ7Ppjvf6QyU1G7rstc=
Encrypted Symmetric Key: fKvCUF00opZMTcuWis1CZFil7A9eNyn37y3k8telqWdm1sSRDywS5MLcCvplBLbVHGKox9aWbokA71fwaCLZ31o0NjsGdfwqhweL3T+nGAEqyuYVjvv4K3OE6cOg0MJS41lVDbIeVgjdGDwregFrOQrnIcRzzYk4zjz0Ao7oraeXCAzGQuZZF8meJU8R4hOqxuhG9BqXslpxiWPs6owJK1NJUdLHpMdbNN1cki5YYw6Mfqiok6vK0cwZrIUhSd7vo1IYUpBH+hMYpV2qjDTdJMbc6g5q7uIPwpNFoLrPAYCueHJhN5EOC2Ynxdfex69tx+YnBjDWRE7clYvulhEUgQ==
Tag: XBQYDhJmndSJGNHoUuNKUQ==
Nonce: I4B8q7CjTHOqd2j3kr670g==
Decrypted Data: Confidential data for group members only.
